In [25]:
import pymongo
import pandas as pd
import json
import os

# Daten einlesen

Da alle csv-Dateien im Ordner 'data' enthalten sind, können wir mit dem Befehl ```os.listdir(PATH)``` alle Dateinamen auflisten und in der Variable *files* speichern.

In [26]:
files = [ 'data/' + x for x in os.listdir('data') ]
files

['data/bremen-obernstraße-20190930-20221031-hour.csv',
 'data/dresden-prager straße (nord)-20190728-20221031-hour.csv',
 'data/düsseldorf-königsallee ostseite (süd)-20180430-20221031-hour.csv',
 'data/frankfurt a.m.-zeil (mitte)-20190630-20221031-hour.csv',
 'data/hamburg-jungfernstieg-20180927-20221031-hour.csv',
 'data/hannover-georgstraße-20180731-20221031-hour.csv',
 'data/mainz-stadthausstraße-20180430-20221031-hour.csv',
 'data/münchen-kaufingerstraße-20190827-20221031-hour.csv',
 'data/rostock-kröpeliner straße (west)-20191031-20221031-hour.csv',
 'data/saarbrücken-bahnhofstraße (mitte)-20180430-20221031-hour.csv',
 'data/stuttgart-königstraße (mitte)-20180430-20221031-hour.csv']

# Daten aufbereiten

Zwar jliegen uns die Daten bereits im csv Format vor und können somit theoretisch direkt verwendet werden, für spätere Auswertungen ist es dennoch sinnvoll einige der Spalten, die aus zusammengesetzten Informationen bestehen, in ihre atomaren Bestandteile aufzuteilen und diese einzeln zu speichern.

Um die Transformation der Daten durchzuführen, wird das Package *Pandas* verwendet. Wir startet mit dem Einlesen der csv Dateien in einzelne Panda-Dataframes.

In [27]:
data = [ pd.read_csv(x, delimiter=';') for x in files ] # Stadt1 in data[0], Stadt2 in data[1] ...
data[0].head()

,location,time of measurement,weekday,pedestrians count,temperature in ºc,weather condition,incidents
0,"Obernstraße, Bremen",2019-10-01 00:00:00 +0200,Tuesday,16,NaN,NaN,NaN
1,"Obernstraße, Bremen",2019-10-01 01:00:00 +0200,Tuesday,7,NaN,NaN,NaN
2,"Obernstraße, Bremen",2019-10-01 02:00:00 +0200,Tuesday,6,NaN,NaN,NaN
3,"Obernstraße, Bremen",2019-10-01 03:00:00 +0200,Tuesday,5,NaN,NaN,NaN
4,"Obernstraße, Bremen",2019-10-01 04:00:00 +0200,Tuesday,4,NaN,NaN,NaN


Aufgespaltet werden demnach also die beiden Spalten **location** und **time of measurement**.

Die Daten der Spalte **location** bestehen jeweils immer aus zwei Informationen: Dem Ort der Messung und die dazugehörige Stadt. Separiert werden diese durch ein Komma. Da unabhängig vom Namen des Ortes oder Stadt in den Zellen jeweils immer nur ein Komma zu finden ist, können wir die von Pandas zur Verfügung gestellte Funktion ```DATAFRAME.str.split()``` verwenden, die die Werte anhand eines gegeben Strings separiert und daraus neue Spalten erstellt.

In [28]:
def separateLocation(data):
    '''Separate location into [location, city]'''
    for i in range(len(data)):
        data[i][['location', 'city']] = data[i]['location'].str.split(",", expand=True)
    return data

Die zweite zu separierende Information liegt in der Spalte **time of measurement**. Hier enthalten sind sowohl nützliche, als auch für uns nicht weiter wichtige Informationen.

Die Spalte selbst besteht wiederum aus drei Informationen: Dem Datum, der Uhrzeit und der Zeitzone, auf welche die Zeitangabe bezogen ist. Diese Daten sind mit Leerzeichen separiert, wir gehen also ähnlich wie mit der Spalte **location** vor und spalten die Information in ihre Bestandteile auf. Um leichter mit den Datums- und Zeitangaben arbeiten zu können, spalten wir diese ebenso auf.

In [29]:
def separateTime(data):
    '''Separate time of measurement into [year, month, day, hour, minute, second, timezone]'''
    for i in range(len(data)):
        data[i][['date','time', 'timezone']]= data[i]["time of measurement"].str.split(" ",expand=True)
        data[i][['year', 'month', 'day']] = data[i]['date'].str.split("-", expand=True)
        data[i][['hour', 'minute', 'second']] = data[i]['time'].str.split(":", expand=True)
    return data

Da wir nur Städte in Deutschland betrachten, ist die Information in welcher Zeitzone die Messung durchgeführt wurde nicht weiter relevant und kann somit entfernt werden. Ebenso interessiert uns die Minute und Sekunde der Messung nicht, da immer nur zur vollen Stunde gemessen wurde. Wir entfernen also für uns nicht weiter relevante Spalte und bennenen zusätzlich diverse Spalten für eine bessere Lesbarkeit um. Schließen werden noch die Datentypen der Spalten angepasst, um leichter mit ihnen arbeiten zu können.

In [31]:
def cleanData(data):
    '''Rename columns, drop unneccessary columns and adjust data types'''
    for i in range(len(data)):
        data[i] = data[i].rename(columns={"pedestrians count" : "pedestrians", "temperature in ºc" : "temperature", "weather condition" : "weatherCondition"})
        data[i].drop(columns=['time of measurement', 'date', 'time', 'timezone', 'minute', 'second'], inplace=True)
        data[i] = data[i].astype({'year' : 'int16', 'month' : 'int16', 'day' : 'int16', 'hour' : 'int16', 'incidents' : 'int16', 'temperature' : 'int16', 'pedestrians' : 'int32'}, errors='ignore')
    return data

Zuletzt wenden wir alle Funktionen auf unseren Datensatz an und erhalten die angepassten Dataframes.

In [32]:
data = separateLocation(data)
data = separateTime(data)
data = cleanData(data)

In [33]:
data[0].head(5)

,location,weekday,pedestrians,temperature,weatherCondition,incidents,city,year,month,day,hour
0,Obernstraße,Tuesday,16,NaN,NaN,NaN,Bremen,2019,10,1,0
1,Obernstraße,Tuesday,7,NaN,NaN,NaN,Bremen,2019,10,1,1
2,Obernstraße,Tuesday,6,NaN,NaN,NaN,Bremen,2019,10,1,2
3,Obernstraße,Tuesday,5,NaN,NaN,NaN,Bremen,2019,10,1,3
4,Obernstraße,Tuesday,4,NaN,NaN,NaN,Bremen,2019,10,1,4


In [35]:
d = list()

for i in range(len(data)):
    
    d.append(data[i][data[i]['weatherCondition'].isna()])

In [37]:
for i in range(len(d)):
    print(d[i])

        location  weekday  pedestrians  temperature weatherCondition  \
0    Obernstraße  Tuesday           16          NaN              NaN   
1    Obernstraße  Tuesday            7          NaN              NaN   
2    Obernstraße  Tuesday            6          NaN              NaN   
3    Obernstraße  Tuesday            5          NaN              NaN   
4    Obernstraße  Tuesday            4          NaN              NaN   
..           ...      ...          ...          ...              ...   
499  Obernstraße   Monday          822          NaN              NaN   
500  Obernstraße   Monday          320          NaN              NaN   
501  Obernstraße   Monday          154          NaN              NaN   
502  Obernstraße   Monday          109          NaN              NaN   
503  Obernstraße   Monday           75          NaN              NaN   

    incidents     city  year  month  day  hour  
0         NaN   Bremen  2019     10    1     0  
1         NaN   Bremen  2019     10  

# Daten in MongoDB speichern

Um die verarbeiteten Daten schließlich dauerhaft zu speichern, lesen wir sie in ein MongoDB ein. Dafür wird das Package *pymongo* verwendet.

Zunächst muss der MongoDB Service gestartet werden, anschließend kann mit der Ausführung des Notebooks fortgeführt werden. Wir startet damit uns mit dem Service zu verbinden.

In [9]:
client = pymongo.MongoClient('mongodb://localhost:27017')

MongoDB ist ein nicht relationale Datenbank, die ihre Daten im JSON-Format hält. Pandas stellt hierfür die Funktion ```DATAFRAME.to_dict()``` zur Verfügung, mit welcher wir unsere Dataframes in ein JSON-Objekt konvertieren können.

In [10]:
dataJson = dict()
for i in range(len(data)):
    data[i] = data[i].to_dict(orient='records')

Nun muss auf die Datenbank zugegriffen werden. Falls diese noch nicht vorhanden ist, wird sie automatisch erstellt. Wir speichern anschließend alle Daten in der Collection **HyStreetData**.

In [11]:
db = client['HyStreet']

for i in range(len(data)):
    db.HyStreetData.insert_many(data[i])